#### exercise 1

In [ ]:
# import libs
import numpy as np
import polars as pl
import pandas as pd

In [ ]:
# read csv to dataframe
pl_ames = pl.read_csv("../AmesHousing.csv")
pl_xlsx = pl.read_excel("../Neighborhood_names.xlsx")

In [ ]:
pl_ames.join(
    pl_xlsx,
    left_on="Neighborhood",
    right_on="Neighborhood",
    how="left",
).write_csv("ames_neighborhoods.csv")

#### exercise 2

In [ ]:
# split dataframe into subsets by type
pl_ames_int = pl_ames.select(pl.col(pl.Int64))
pl_ames_string = pl_ames.select(pl.col(pl.String))

In [ ]:
# show all distinct types
print(set(pl_ames.dtypes))

# count number of each type
number_int = pl_ames.dtypes.count(pl.Int64)
number_string = pl_ames.dtypes.count(pl.String)

print(f"Number of int columns: {number_int}, Number of string columns: {number_string}")

In [ ]:
# find all columns with missing values, and show how many
pl_ames.null_count().transpose(include_header=True).filter(pl.col("column_0") > 0).transpose()

In [ ]:
# is sales price complete
pl_ames["SalePrice"].is_null().any()

In [ ]:
pd_ames = pd.read_csv("../AmesHousing.csv")
pd_xlsx = pd.read_excel("../Neighborhood_names.xlsx")

In [ ]:
pd_ames.isnull().sum()

In [ ]:
# construct summary statistics
pl_ames_int.mean()
pl_ames_int.median()
pl_ames_int.std()

In [ ]:
from collections import Counter

def summary_statistics(df):
    for col in df.columns:
        print(f"Summary statistics for {col}:")
        print(f"Mean: {df[col].n_unique()}\n")
        print(f"Median: {df[col].mode()[0]}\n")
        print(f"Frequency: {Counter(df[col])}\n")

summary_statistics(pl_ames_string)

#### exercise 3

In [ ]:
# reaplce NA with other in string columns
pl_ames_string = pl_ames_string.with_columns([
    pl.col(col).str.replace("NA", "other") for col in pl_ames_string.columns
])

pl_ames_int_filled = pl_ames_int.fill_null(strategy="mean")
pl_ames_string_filled = pl_ames_string.fill_null("other")

In [ ]:
pl_ames_recombined = pl.concat([pl_ames_int_filled, pl_ames_string_filled], how="horizontal")

In [ ]:
# cast all columns to string
""" pl_ames_int_dc = pl_ames_int_filled.with_columns([
    pl.col(col).cast(pl.Int32) for col in pl_ames_int_filled.columns
]) """

def f_pl_downcast_numerical_column(df, s_col):

    # Assign Polars series to object ps_col.
    ps_col = df[s_col]

    # If the data type is Integer.
    if ps_col.dtype in [pl.Int8, pl.Int16, pl.Int32, pl.Int64]:

        # Determine min and max value.
        n_min = ps_col.min()
        n_max = ps_col.max()
        
        # If all values in ps_col are positive.
        if n_min >= 0:
            if n_max <= 255:
                return pl.col(s_col).cast(pl.UInt8)
            elif n_max <= 65535:
                return pl.col(s_col).cast(pl.UInt16)
            elif n_max <= 4294967295:
                return pl.col(s_col).cast(pl.UInt32)
            else:
                return pl.col(s_col).cast(pl.UInt64)
            
        # If one or more values are negative. 
        else:
            if n_min >= -128 and n_max <= 127:
                return pl.col(s_col).cast(pl.Int8)
            elif n_min >= -32768 and n_max <= 32767:
                return pl.col(s_col).cast(pl.Int16)
            elif n_min >= -2147483648 and n_max <= 2147483647:
                return pl.col(s_col).cast(pl.Int32)
            else:
                return pl.col(s_col).cast(pl.Int64)
            
    # If the data type is Float64. Note, Polars accepts
    # Float32 and Float64.
    elif ps_col.dtype == pl.Float64:
        return pl.col(s_col).cast(pl.Float32)
    
    # In all other cases.
    else:
        return pl.col(s_col)
    
# Apply the downcasting
pl_ames_int_dc = pl_ames_int_filled.with_columns([

    # Calling the function above on each column in df_pl_reduced.
    f_pl_downcast_numerical_column(pl_ames_int_filled, s_col).alias(s_col)

    # Looping through all columns.
    # Why can we leave in the categorical columns?
    for s_col in pl_ames_int_filled.columns
])

pl_ames_str_dc = pl_ames_string_filled.with_columns([
    pl.col(col).cast(pl.Categorical) for col in pl_ames_string_filled.columns
])

pl_ames_recombined_dc = pl.concat([pl_ames_int_dc, pl_ames_str_dc], how="horizontal")

# size comparison
for i in [
        pl_ames, pl_ames_recombined, pl_ames_recombined_dc,
        pl_ames_int, pl_ames_int_filled, pl_ames_int_dc,
        pl_ames_string, pl_ames_string_filled, pl_ames_str_dc
    ]:
    print( i.estimated_size() )

print(f"Downcast size is only {round(pl_ames_recombined_dc.estimated_size()/pl_ames.estimated_size(), 2)} times the size of the original.")

#### Exercise 4

In [ ]:
# a
pl_ames["SalePrice"].describe()

In [ ]:
# b
import altair as alt
# plot histogram of SalePrice
alt.Chart(pl_ames.to_pandas()).mark_bar().encode(
    alt.X("SalePrice", bin=alt.Bin(maxbins=50)),
    alt.Y("count()"),
    tooltip=["SalePrice", "count()"]
).properties(
    title="Histogram of Sale Prices"
).interactive()

In [ ]:
# c
# scatter plot of SalePrice vs GrLivArea
alt.Chart(pl_ames.to_pandas()).mark_circle(size=60).encode(
    x='Gr Liv Area',
    y='SalePrice',
    tooltip=['Gr Liv Area', 'SalePrice']
).interactive()

In [ ]:
# d
# box plot of SalePrice by Neighborhood
alt.Chart(pl_ames.to_pandas()).mark_boxplot().encode(
    x='Neighborhood',
    y='SalePrice',
    tooltip=['Neighborhood', 'SalePrice']
).properties(
    title="Box Plot of Sale Prices by Neighborhood"
).interactive()

In [ ]:
# e
""" 
most home prices live between 100k and 200k, with a few outliers above 500k.
"""

In [ ]:
# f
""" 
assessing the boxplot of SalePrice by Neighborhood 

some neighborhoods have a wider range of SalePrices, indicating more variability in home prices.
other neighborhoods have a narrower range, suggesting more consistent home prices, but also show more outliers.

since some neighbourhoods have a widers spread of SalePrices, the model might not perform well across these neighborhoods.
training on these neightbourhoods can lead to overfitting.

removing outliers might help improve the model's performance, but means we get a narrower view of the data.

"""

In [ ]:
# g
import matplotlib.pyplot as plt
import seaborn as sns
# scatter plot of salerice vs 5 columns
columns_to_plot = ["Gr Liv Area", "Total Bsmt SF", "Garage Area", "Lot Area", "Year Built"]
for col in columns_to_plot:
    sns.scatterplot(data=pl_ames.to_pandas(), x=col, y="SalePrice")
    sns.regplot(data=pl_ames.to_pandas(), x=col, y="SalePrice", scatter=False, color='red')
    plt.title(f'SalePrice vs {col}')
    plt.xlabel(col)
    plt.ylabel('SalePrice')
    plt.show()
    

In [ ]:
# h
# pearson correlation matrix for numerical columns
correlation_matrix = pl_ames_int.corr()
# convert to pandas for better visualization
correlation_matrix_pd = correlation_matrix.to_pandas()
# show table
print(correlation_matrix_pd)

In [ ]:
# i
# correlation matrix heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix_pd, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8})
plt.title('Correlation Matrix Heatmap')

#### Exercise 5

In [ ]:
# a 
# linear regression model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
# prepare data
X = pl_ames_int_filled.drop("SalePrice").to_pandas()
y = pl_ames_int_filled["SalePrice"].to_pandas()
# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)
# create linear regression model
model = LinearRegression()
# fit model
model.fit(X_train, y_train)
# predict on test set
y_pred = model.predict(X_test)
# evaluate model
from sklearn.metrics import root_mean_squared_error, r2_score
mse = root_mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')


In [ ]:
# b 
# lasso regression model
from sklearn.linear_model import Lasso
# create lasso regression model
lasso_model = Lasso(alpha=0.1)
# fit model
lasso_model.fit(X_train, y_train)
# predict on test set
y_lasso_pred = lasso_model.predict(X_test)
# evaluate model
lasso_mse = root_mean_squared_error(y_test, y_lasso_pred, squared=False)
lasso_r2 = r2_score(y_test, y_lasso_pred)
print(f'Lasso Mean Squared Error: {lasso_mse}')
print(f'Lasso R^2 Score: {lasso_r2}')

In [ ]:
# c
# knn regression model
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import root_mean_squared_log_error
# create knn regression model
knn_model = KNeighborsRegressor(n_neighbors=5)
# fit model
knn_model.fit(X_train, y_train)
# predict on test set
y_knn_pred = knn_model.predict(X_test)
# evaluate model using root mean squared error
knn_mse = root_mean_squared_error(y_test, y_knn_pred)
knn_rmsle = root_mean_squared_log_error(y_test, y_knn_pred)
knn_r2 = r2_score(y_test, y_knn_pred)
print(f'KNN Mean Squared Error: {knn_mse}')
print(f'KNN R^2 Score: {knn_r2}')
# 6
print(f'KNN Root Mean Squared Log Error: {knn_rmsle}')